# Model with Feedback

Example testing a model with both predict and feedback in python. This could be used as a basis for a reinforcement learning model deployment.
 

# REST

In [ ]:
!s2i build -E environment_rest . seldonio/seldon-core-s2i-python3:0.3 model-with-feedback-rest:0.1

In [ ]:
!docker run --name "model-with-feedback" -d --rm -p 5000:5000 model-with-feedback-rest:0.1

In [ ]:
!cd ../../../wrappers/testing && make build_protos

## Test predict

In [ ]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p

## Test feedback

In [ ]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p --endpoint send-feedback

In [ ]:
!docker rm model-with-feedback --force

# gRPC

In [ ]:
!s2i build -E environment_grpc . seldonio/seldon-core-s2i-python3:0.3 model-with-feedback-grpc:0.1

In [ ]:
!docker run --name "model-with-feedback" -d --rm -p 5000:5000 model-with-feedback-grpc:0.1

In [ ]:
!cd ../../../wrappers/testing && make build_protos

## Test predict

In [ ]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p --grpc

## Test feedback

In [ ]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p --endpoint send-feedback --grpc

In [ ]:
!docker rm model-with-feedback --force

# Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [ ]:
!minikube start --vm-driver kvm2 --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!helm init

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core 

# REST

In [ ]:
!eval $(minikube docker-env) && s2i build -E environment_rest . seldonio/seldon-core-s2i-python3:0.3 model-with-feedback-rest:0.1 --loglevel 5

In [ ]:
!kubectl create -f deployment-rest.json

Wait until ready (replicas == replicasAvailable)

In [ ]:
!kubectl get seldondeployments mymodel -o jsonpath='{.status}' 

In [ ]:
!cd ../../../util/api_tester && make build_protos 

## Test predict

In [ ]:
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

## Test feedback

In [ ]:
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p --endpoint send-feedback

In [ ]:
!kubectl delete -f deployment-rest.json

# gRPC

In [ ]:
!eval $(minikube docker-env) && s2i build -E environment_grpc . seldonio/seldon-core-s2i-python3:0.3 model-with-feedback-grpc:0.1

In [ ]:
!kubectl create -f deployment-grpc.json

Wait until ready (replicas == replicasAvailable)

In [ ]:
!kubectl get seldondeployments mymodel -o jsonpath='{.status}' 

In [ ]:
!cd ../../../util/api_tester && make build_protos 

## Test predict

In [ ]:
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[1].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p --grpc --oauth-port `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'`

## Test feedback

In [ ]:
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[1].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p --endpoint send-feedback --grpc --oauth-port `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'`

In [ ]:
!kubectl delete -f deployment-grpc.json

In [ ]:
!minikube delete